In [1]:
from hmmlearn import hmm
from hmmlearn.base import _log
import numpy as np
import numpy.typing as npt
from scipy.stats.qmc import LatinHypercube
# import tensorflow as tf
import torch
from discretized_HMM import DiscreteHMM, HmmOptim

In [17]:
HMM_model = hmm.GaussianHMM(3).fit(np.random.multivariate_normal(np.array([0, 1]), np.array([[1, 0],  [0,1]]), 100))
myHMM = DiscreteHMM('random', 10, 3, learning_alg="em")
myHMM2 = DiscreteHMM('uniform', 10, 3)
myHMM3 = DiscreteHMM('latin_cube_u', 10, 3)
myHMM4 = DiscreteHMM('latin_cube_q', 10, 3)
obs, hid = HMM_model.sample(100)
myHMM.fit(obs)
myHMM2.fit(obs)
myHMM3.fit(obs)
myHMM4.fit(obs)

/home/kabalce/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/kabalce/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Optimizing separately attribute 'startprob_' ignores the stationarity requirement


In [18]:
myHMM

DiscreteHMM(l=3, learning_alg='em', no_nodes=10)

In [25]:
HMM_model.covars_[0]

array([[0.19427891, 0.        ],
       [0.        , 0.86494671]])

In [37]:
dist

MultivariateNormal(loc: torch.Size([3, 2]), covariance_matrix: torch.Size([3, 2, 2]))

In [55]:
dists = [torch.distributions.MultivariateNormal(torch.tensor(HMM_model.means_[i]), torch.tensor(HMM_model.covars_[i])) for i in range(HMM_model.means_.shape[0])]
probs = torch.cat([dist.log_prob(torch.Tensor(myHMM.nodes.T)).reshape(-1,1) for dist in dists], dim=1)
probs = torch.nn.functional.normalize(probs, dim=0)

In [56]:
probs.shape

torch.Size([10, 3])

In [57]:
probs

tensor([[-0.2443, -0.3074, -0.2651],
        [-0.2026, -0.1473, -0.1802],
        [-0.0876, -0.2243, -0.1286],
        [-0.0768, -0.1721, -0.1359],
        [-0.1609, -0.1067, -0.1909],
        [-0.1198, -0.3852, -0.1719],
        [-0.1054, -0.2935, -0.1849],
        [-0.8669, -0.2269, -0.4381],
        [-0.1092, -0.7116, -0.4166],
        [-0.2662, -0.0124, -0.6297]], dtype=torch.float64)

In [11]:
myHMM.nodes.shape

(1, 10)

In [12]:
HMM_model.covars_.shape

(3, 1, 1)

In [61]:
S = np.log(HMM_model.transmat_ * HMM_model.startprob_)

In [63]:
B = probs

In [65]:
B @ S @ B.T

tensor([[-17.1572, -12.7818,  -7.4234,  -7.1799, -11.6181, -10.1808,  -9.9181,
         -43.5181, -18.0881, -28.6921],
        [-11.1973,  -8.3467,  -4.8408,  -4.6789,  -7.5794,  -6.6401,  -6.4640,
         -28.4663, -11.7667, -18.6749],
        [ -9.1874,  -6.8394,  -3.9791,  -3.8520,  -6.2246,  -5.4559,  -5.3202,
         -23.2350,  -9.7257, -15.4185],
        [ -8.0378,  -5.9843,  -3.4808,  -3.3686,  -5.4444,  -4.7734,  -4.6530,
         -20.3401,  -8.5019, -13.4742],
        [ -9.6832,  -7.2175,  -4.1869,  -4.0463,  -6.5534,  -5.7439,  -5.5906,
         -24.6150, -10.1774, -16.1424],
        [-14.0848, -10.4829,  -6.1019,  -5.9090,  -9.5450,  -8.3654,  -8.1605,
         -35.5867, -14.9296, -23.6692],
        [-12.1706,  -9.0595,  -5.2719,  -5.1035,  -8.2456,  -7.2286,  -7.0489,
         -30.7720, -12.8884, -20.4271],
        [-32.6768, -24.3851, -14.1039, -13.6183, -22.1071, -19.3492, -18.8156,
         -83.4172, -34.1343, -54.2618],
        [-25.6576, -19.0875, -11.1235, -10.7743,

In [72]:
torch.softmax(torch.tensor(np.exp(S)), dim=1)

tensor([[0.3237, 0.3527, 0.3237],
        [0.3208, 0.3583, 0.3208],
        [0.3248, 0.3504, 0.3248]], dtype=torch.float64)

In [76]:
sum([0.3248, 0.3208, 0.3237])

0.9692999999999999

In [77]:
covar = HMM_model.covars_[0]

In [79]:
np.linalg.cholesky(covar)

array([[0.44077081, 0.        ],
       [0.        , 0.93002511]])

In [80]:
n_components = 3
n_dim = 2
np.tril(np.random.standard_normal(n_components * n_dim ** 2).reshape(n_components, n_dim, n_dim))

array([[[ 0.53985629,  0.        ],
        [-0.29998013, -0.44768971]],

       [[-0.00227077,  0.        ],
        [ 0.10262521,  0.87128949]],

       [[ 0.14473708,  0.        ],
        [-0.31126927,  0.71334796]]])

In [84]:
transmat = HMM_model.transmat_ / HMM_model.transmat_.sum(axis=1)[:, np.newaxis]

In [86]:
transmat

array([[0.00132856, 0.0859425 , 0.91272894],
       [0.36666489, 0.1105509 , 0.52278421],
       [0.03373398, 0.07595588, 0.89031014]])